##  Price a maximum rainbow option
### 1. Monte Carlo simulatiton and Cholesky decomposition method
### 2. Combine the antithetic variate approach and moment matching method
### 3. Implement the inverse Cholesky method in Wang (2008) to price the above rainbow option.



## Monte Carlo simulatiton and Cholesky decomposition method

In [57]:
# Price a maximum rainbow option with Monte Carlo simulatiton and Cholesky decomposition method
# Input
import numpy as np
import math

# input 
k = 100 # Strike price
r = 0.1
t = 0.5
size = 10000 # number of simulations
repetitions = 20 # number of repetitions
n = 5 # number of underlying stocks
pl = [95,95,95,95,95] # list of current prices
q = [0.05,0.05,0.05,0.05,0.05]
sdl = [0.5,0.5,0.5,0.5,0.5] # list of sigma
cor_coe = [[1.0,0.5,0.5,0.5,0.5],
           [0.5,1.0,0.5,0.5,0.5],
          [0.5,0.5,1.0,0.5,0.5],
           [0.5,0.5,0.5,1.0,0.5],
           [0.5,0.5,0.5,0.5,1.0]]
# INPUT SPACE END
# input space end

mean = []
var = []
for i in range(n):
    mean.append(math.log(pl[i])+(r-q[i]-((sdl[i]**2)/2))*t)
    var.append((sdl[i]**2)*t)


# data process - 1.Calculate Covariance array with sigma and correlation coefficient
c = np.zeros_like(cor_coe)
for i in range(n):
    for j in range(n):
        c[i,j] = float(sdl[i] * sdl[j] * cor_coe[i][j]*t)
print("covariance matrix ")
print(c)
# Calculation - Transform Covariance array into array A with Cholesky Decomposition
# c = [[4,2,6],[2,5,2],[6,7,14]]
# c = np.array(c)
a = np.zeros_like(c)
a[0,0] = np.sqrt(c[0,0]) # step 1
for i in range(n):
    for j in range(i + 1):
        if i == j:
            v = c[i,i] - np.sum(np.square(a[i,:i]))  # step 2
            if v<0:
                v = 0
            a[i,i] = np.sqrt(v)
        else:
            a[i,j] = (c[i,j] - np.sum(a[i,:j] * a[j,:j])) / a[j,j]  # step 3, step 4
print("A ")
print(a.T)

# data process - Price a maximum rainbow option with Monte Carlo simulatiton
ans_c = []
ans_p = []
rlpp = []
for rep in range(repetitions):
    rlc = []
    rlp = []
    ansc = []
    ansp = []
    zl = [] # list of random samples (number of simulations *n)
    for i in range(size):
        zl.append(np.random.standard_normal(n))
    rlc = np.dot(zl,a.T) # np.dot(Array of random samples, array A ) 
    for i in range(len(rlc)):
        rlp.append(rlc[i])
    rlp = np.array(rlp)
    #print(rlc,rlp)
    # call
    for i in range(size):
        for j in range(n):
            val = np.exp(mean[j]+rlc[i][j])
            if val - k < 0:
                rlc[i][j] = 0
            else:
                rlc[i][j] = val - k
    # put
    for i in range(size):
        for j in range(n):
            val = np.exp(mean[j]+rlp[i][j])
            if k - val < 0:
                rlp[i][j] = 0
            else:
                rlp[i][j] = k - val
    for i in range(size):
        ansc.append(max(rlc[i,:]))
        ansp.append(max(rlp[i,:]))
    ans_c.append(np.mean(ansc)*np.exp(-r*t))
    ans_p.append(np.mean(ansp)*np.exp(-r*t))

ans_meanc = np.mean(ans_c)
ans_stdc = np.std(ans_c)
ans_meanp = np.mean(ans_p)
ans_stdp = np.std(ans_p)
print("Calculate Price of European Option by Monte Carlo")
print("call price : ",round(ans_meanc,4))
print("95% interval of call price : ",ans_meanc- 2*ans_stdc,ans_meanc+2*ans_stdc)
print("put price : ",round(ans_meanp,4))
print("95% interval of put price : ",ans_meanp- 2*ans_stdp,ans_meanp+2*ans_stdp)

covariance matrix 
[[0.125  0.0625 0.0625 0.0625 0.0625]
 [0.0625 0.125  0.0625 0.0625 0.0625]
 [0.0625 0.0625 0.125  0.0625 0.0625]
 [0.0625 0.0625 0.0625 0.125  0.0625]
 [0.0625 0.0625 0.0625 0.0625 0.125 ]]
A 
[[0.35355339 0.1767767  0.1767767  0.1767767  0.1767767 ]
 [0.         0.30618622 0.10206207 0.10206207 0.10206207]
 [0.         0.         0.28867513 0.07216878 0.07216878]
 [0.         0.         0.         0.2795085  0.0559017 ]
 [0.         0.         0.         0.         0.27386128]]
Calculate Price of European Option by Monte Carlo
call price :  30.4355
95% interval of call price :  29.854118652268635 31.01694256268723
put price :  28.5922
95% interval of put price :  28.27019036731743 28.914159717212915


## Combine the antithetic variate approach and moment matching method


In [43]:
# Combine the antithetic variate approach and moment matching method
# Input
import numpy as np
import math

# INPUT 
k = 100 # Strike price
r = 0.1
t = 0.5
size = 10000 # number of simulations
repetitions = 20 # number of repetitions
n = 5 # number of underlying stocks
pl = [95,95,95,95,95] # list of current prices
q = [0.05,0.05,0.05,0.05,0.05]
sdl = [0.5,0.5,0.5,0.5,0.5] # list of sigma
cor_coe = [[1.0,0.5,0.5,0.5,0.5],
           [0.5,1.0,0.5,0.5,0.5],
          [0.5,0.5,1.0,0.5,0.5],
           [0.5,0.5,0.5,1.0,0.5],
           [0.5,0.5,0.5,0.5,1.0]]
# INPUT SPACE END

mean = []
var = []
for i in range(n):
    mean.append(math.log(pl[i])+(r-q[i]-((sdl[i]**2)/2))*t)
    var.append((sdl[i]**2)*t)


# data process - 1.Calculate Covariance array with sigma and correlation coefficient
c = np.zeros_like(cor_coe)
for i in range(n):
    for j in range(n):
        c[i,j] = float(sdl[i] * sdl[j] * cor_coe[i][j]*t)
print("covariance matrix")
print(c)
# Calculation - Transform Covariance array into array A with Cholesky Decomposition
# c = [[4,2,6],[2,5,2],[6,7,14]]
# c = np.array(c)
a = np.zeros_like(c)
a[0,0] = np.sqrt(c[0,0]) # step 1
for i in range(n):
    for j in range(i + 1):
        if i == j:
            v = c[i,i] - np.sum(np.square(a[i,:i]))  # step 2
            if v<0:
                v = 0
            a[i,i] = np.sqrt(v)
        else:
            a[i,j] = (c[i,j] - np.sum(a[i,:j] * a[j,:j])) / a[j,j]  # step 3, step 4
print("A")
print(a.T)
# data process - Price a maximum rainbow option with Monte Carlo simulatiton
ans_c = []
ans_p = []
std_r = []
for rep in range(repetitions):
    rlc = []
    rlp = []
    ansc = []
    ansp = []
    zl = [] # list of random samples (number of simulations *n)
    for i in range(int(size/2)):
        zl.append(np.random.standard_normal(n))  #   antithetic variate approach 

    for i in range(int(size/2)):
        zl.append([])
        for j in range(n):
            zl[int(size/2)+i].append(-zl[i][j])

    zl = np.array(zl)
    std_r = []
    for i in range(n):
        std_r.append(np.std(zl[:,i]))   #   moment matching
    for i in range(size):
        for j in range(n):
            zl[i][j] = zl[i][j]/std_r[j]
    #print("std",np.std(zl[:,0]),np.std(zl[:,1]),np.std(zl[:,2]))
    zl = np.array(zl)
    
    rlc = np.dot(zl,a.T) # np.dot(Array of random samples, array A ) 
    for i in range(len(rlc)):
        rlp.append(rlc[i])
    rlp = np.array(rlp)
    #print(rlc,rlp)
    # call
    for i in range(size):
        for j in range(n):
            val = np.exp(mean[j]+rlc[i][j])
            if val - k < 0:
                rlc[i][j] = 0
            else:
                rlc[i][j] = val - k
    # put
    for i in range(size):
        for j in range(n):
            val = np.exp(mean[j]+rlp[i][j])
            if k - val < 0:
                rlp[i][j] = 0
            else:
                rlp[i][j] = k - val
    for i in range(size):
        ansc.append(max(rlc[i,:]))
        ansp.append(max(rlp[i,:]))
    ans_c.append(np.mean(ansc)*np.exp(-r*t))
    ans_p.append(np.mean(ansp)*np.exp(-r*t))

ans_meanc = np.mean(ans_c)
ans_stdc = np.std(ans_c)
ans_meanp = np.mean(ans_p)
ans_stdp = np.std(ans_p)
print("Calculate Price of European Option by Monte Carlo")
print("call price : ",round(ans_meanc,4))
print("95% interval of call price : ",ans_meanc- 2*ans_stdc,ans_meanc+2*ans_stdc)
print("put price : ",round(ans_meanp,4))
print("95% interval of put price : ",ans_meanp- 2*ans_stdp,ans_meanp+2*ans_stdp)

covariance matrix
[[0.125  0.0625 0.0625 0.0625 0.0625]
 [0.0625 0.125  0.0625 0.0625 0.0625]
 [0.0625 0.0625 0.125  0.0625 0.0625]
 [0.0625 0.0625 0.0625 0.125  0.0625]
 [0.0625 0.0625 0.0625 0.0625 0.125 ]]
A
[[0.35355339 0.1767767  0.1767767  0.1767767  0.1767767 ]
 [0.         0.30618622 0.10206207 0.10206207 0.10206207]
 [0.         0.         0.28867513 0.07216878 0.07216878]
 [0.         0.         0.         0.2795085  0.0559017 ]
 [0.         0.         0.         0.         0.27386128]]
Calculate Price of European Option by Monte Carlo
call price :  30.3733
95% interval of call price :  30.200756661494715 30.545783637871963
put price :  28.6074
95% interval of put price :  28.515986775627347 28.69880375373824


## Implement the inverse Cholesky method in Wang (2008) to price the above rainbow option.

In [56]:
# Combine the antithetic variate approach and moment matching method 
# Implement the inverse Cholesky method in Wang (2008) to price the above rainbow option.
# Input
import numpy as np
import math

# INPUT 
k = 100 # Strike price
r = 0.1
t = 0.5
size = 10000 # number of simulations
repetitions = 20 # number of repetitions
n = 5 # number of underlying stocks
pl = [95,95,95,95,95] # list of current prices
q = [0.05,0.05,0.05,0.05,0.05]
sdl = [0.5,0.5,0.5,0.5,0.5] # list of sigma
cor_coe = [[1.0,0.5,0.5,0.5,0.5],
           [0.5,1.0,0.5,0.5,0.5],
          [0.5,0.5,1.0,0.5,0.5],
           [0.5,0.5,0.5,1.0,0.5],
           [0.5,0.5,0.5,0.5,1.0]]
# INPUT SPACE END

mean = []
var = []
for i in range(n):
    mean.append(math.log(pl[i])+(r-q[i]-((sdl[i]**2)/2))*t)
    var.append((sdl[i]**2)*t)


# data process - 1.Calculate Covariance array with sigma and correlation coefficient
c = np.zeros_like(cor_coe)
for i in range(n):
    for j in range(n):
        c[i,j] = float(sdl[i] * sdl[j] * cor_coe[i][j]*t)
print("covariance matrix")
print(c)
# Calculation - Transform Covariance array into array A with Cholesky Decomposition
# c = [[4,2,6],[2,5,2],[6,7,14]]
# c = np.array(c)
a = np.zeros_like(c)
a[0,0] = np.sqrt(c[0,0]) # step 1
for i in range(n):
    for j in range(i + 1):
        if i == j:
            v = c[i,i] - np.sum(np.square(a[i,:i]))  # step 2
            if v<0:
                v = 0
            a[i,i] = np.sqrt(v)
        else:
            a[i,j] = (c[i,j] - np.sum(a[i,:j] * a[j,:j])) / a[j,j]  # step 3, step 4
print("A")
print(a.T)
#print(np.linalg.cholesky(c).T)
# data process - Price a maximum rainbow option with Monte Carlo simulatiton
ans_c = []
ans_p = []
std_r = []
for rep in range(repetitions):
    rlc = []
    rlp = []
    ansc = []
    ansp = []
    zl = [] # list of random samples (number of simulations *n)
    for i in range(int(size/2)):
        zl.append(np.random.standard_normal(n))  #   antithetic variate approach 

    for i in range(int(size/2)):
        zl.append([])
        for j in range(n):
            zl[int(size/2)+i].append(-zl[i][j])

    zl = np.array(zl)
    std_r = []
    for i in range(n):
        std_r.append(np.std(zl[:,i]))   #   moment matching

    for i in range(size):
        for j in range(n):
            zl[i][j] = zl[i][j]/std_r[j]
#     print("std",np.std(zl[:,0]),np.std(zl[:,1]),np.std(zl[:,4]))
#     print("mean",np.mean(zl[:,0]),np.mean(zl[:,1]),np.mean(zl[:,4]))
    zl = np.array(zl)
    
    c2 = np.cov(zl.T)# variance covariance list of zl
#     print("c2")
#     print(c2)
    b = np.zeros_like(c2)
    
    # inverse Cholesky method
    b[0,0] = np.sqrt(c2[0,0]) # step 1
    for i in range(n):
        for j in range(i + 1):
            if i == j:
                v = c2[i,i] - np.sum(np.square(b[i,:i]))  # step 2
                if v<0:
                    v = 0
                b[i,i] = np.sqrt(v)
            else:
                b[i,j] = (c2[i,j] - np.sum(b[i,:j] * b[j,:j])) / b[j,j]  # step 3, step 4

    b_inv = np.linalg.inv(b.T)
#     print("B ")
#     print(b.T)
#     print("B Inverse")
#     print(b_inv)
#     print("B")
#     print(b_inv)
    a2 = np.dot(b_inv,a.T)
    #print(a2)
    rlc = np.dot(zl,a2) # np.dot(Array of random samples, array A ) 
    for i in range(len(rlc)):
        rlp.append(rlc[i])
    rlp = np.array(rlp)
    #print(rlc,rlp)
    # call
    for i in range(size):
        for j in range(n):
            val = np.exp(mean[j]+rlc[i][j])
            if val - k < 0:
                rlc[i][j] = 0
            else:
                rlc[i][j] = val - k
    # put
    for i in range(size):
        for j in range(n):
            val = np.exp(mean[j]+rlp[i][j])
            if k - val < 0:
                rlp[i][j] = 0
            else:
                rlp[i][j] = k - val
    for i in range(size):
        ansc.append(max(rlc[i,:]))
        ansp.append(max(rlp[i,:]))
    ans_c.append(np.mean(ansc)*np.exp(-r*t))
    ans_p.append(np.mean(ansp)*np.exp(-r*t))

ans_meanc = np.mean(ans_c)
ans_stdc = np.std(ans_c)
ans_meanp = np.mean(ans_p)
ans_stdp = np.std(ans_p)
print("Calculate Price of European Option by Monte Carlo")
print("call price : ",round(ans_meanc,4))
print("95% interval of call price : ",ans_meanc- 2*ans_stdc,ans_meanc+2*ans_stdc)
print("put price : ",round(ans_meanp,4))
print("95% interval of put price : ",ans_meanp- 2*ans_stdp,ans_meanp+2*ans_stdp)

covariance matrix
[[0.125  0.0625 0.0625 0.0625 0.0625]
 [0.0625 0.125  0.0625 0.0625 0.0625]
 [0.0625 0.0625 0.125  0.0625 0.0625]
 [0.0625 0.0625 0.0625 0.125  0.0625]
 [0.0625 0.0625 0.0625 0.0625 0.125 ]]
A
[[0.35355339 0.1767767  0.1767767  0.1767767  0.1767767 ]
 [0.         0.30618622 0.10206207 0.10206207 0.10206207]
 [0.         0.         0.28867513 0.07216878 0.07216878]
 [0.         0.         0.         0.2795085  0.0559017 ]
 [0.         0.         0.         0.         0.27386128]]
Calculate Price of European Option by Monte Carlo
call price :  30.3899
95% interval of call price :  30.245150471425813 30.534628012269398
put price :  28.5998
95% interval of put price :  28.52586999282725 28.673810767550467
